# Project 1 - Explanitory visualization 
## Information Visualization 

### Datacleaning

In [1]:
import altair as alt
import pandas as pd
import numpy as np

First we have to investigate the data we have been provided. Therfore, the three files are loaded using pandas, and then they are displayed as raw data first, only taking the seperator into account. 

In [2]:

#Data overview of each file
df_cruz = pd.read_csv('cruz_list.csv')
display(df_cruz.head())


,grant_number;in_cruz_list
0,2011780;FALSE
1,2027519;FALSE
2,2034824;FALSE
3,2040209;FALSE
4,2040684;FALSE


In this file, it is noticed that the columns are not seperated correctly and that the values for true and false are strings and not actually booleans. Furthermore the grant number also has to be defined as an integer. Furthermore the values in gran_number is the same as grant_id in df_nsf, so by renaming the column, it can be merged. 

In [3]:
df_flagged = pd.read_csv('flagged_words_trump_admin.csv')
display(df_flagged.head())


,flagged_word
0,activism
1,activists
2,advocacy
3,advocate
4,barrier


This file contains data of flagged words, and are not related to a key column. As it is only a file containing a column with the string of words that are flagged, it is possible to implement this data into the merged dataset as booleans. 

In [4]:
df_nsf = pd.read_csv('nsf_terminations_airtable.csv')
display(df_nsf.head())

,grant_id,status,terminated,suspended,termination_date,reinstated,reinstatement_date,reinstatement_indicator,nsf_url,usaspending_url,...,usaspending_obligated,usaspending_outlaid,estimated_budget,estimated_outlays,estimated_remaining,division,directorate,div,dir,record_sha1
0,2201796,❌ Terminated,True,False,2025-04-18,False,NaN,NaN,https://www.nsf.gov/awardsearch/showAward?AWD_...,https://www.usaspending.gov/award/ASST_NON_220...,...,1083218.0,901981.05,1083218,9.019811e+05,1.812369e+05,Research on Learning in Formal and Informal Se...,STEM Education,DRL,EDU,5866299f5d416a23e01823f57f857aad2950cd4d
1,2315095,❌ Terminated,True,False,2025-04-18,False,NaN,NaN,https://www.nsf.gov/awardsearch/showAward?AWD_...,https://www.usaspending.gov/award/ASST_NON_231...,...,NaN,NaN,3641116,3.641116e+06,0.000000e+00,NaN,NaN,NaN,NaN,136bbe81751eb4f4ae26f6dceb68ee1339877daa
2,2236163,❌ Terminated,True,False,2025-04-18,False,NaN,NaN,https://www.nsf.gov/awardsearch/showAward?AWD_...,https://www.usaspending.gov/award/ASST_NON_223...,...,609429.0,348712.21,609429,3.487122e+05,2.607168e+05,Undergraduate Education,STEM Education,DUE,EDU,075e2156a2996eba5595f0095b20da6da75c06e7
3,2243822,🔄 Possibly Reinstated,True,False,2025-04-18,True,2025-06-30,Thakur v. Trump (Jun 2025),https://www.nsf.gov/awardsearch/showAward?AWD_...,https://www.usaspending.gov/award/ASST_NON_224...,...,NaN,NaN,3000000,1.079409e+06,1.920591e+06,NaN,NaN,NaN,NaN,f9a85b61f807953ecdb2a77c92362c042dcfb494
4,2216648,❌ Terminated,True,False,2025-04-18,False,NaN,NaN,https://www.nsf.gov/awardsearch/showAward?AWD_...,https://www.usaspending.gov/award/ASST_NON_221...,...,2719700.0,1547300.07,2719700,1.547300e+06,1.172400e+06,Biological Infrastructure,Biological Sciences,DBI,BIO,f35dec9f84e3160f0ff2eee79200f25ff5471f67


The first thing that pops into the eye is the status column,  which could be made into a binary value, and that way not disurb the eye as much as the raw data and it could also be handy in the further analysis. Furthermore there are 35 columns, and by looking at the question in the project, it is not nescessary to keep them all. Therfore an array with the relevant comlums are made. It is also recognised that there is a shared column with the fist datafile containing the grant_id, and therefore the two can be merged. Below are the commands described for each datafile done. 

In [5]:
#data treatment for cruz_list.csv
df_cruz[['grant_number', 'in_cruz_list']] = df_cruz['grant_number;in_cruz_list'].str.split(';', expand=True)
df_cruz['grant_id'] = df_cruz['grant_number'].astype(int)
df_cruz['in_cruz_list'] = df_cruz['in_cruz_list'].map({'TRUE': True, 'FALSE': False})


#Data treatment for nsf.csv
df_nsf['status'] = df_nsf['status'].map({'❌ Terminated': 0, '🔄 Possibly Reinstated': 1})

keep = [
    'grant_id',                  # key column
    'org_name',                  # institution name
    'org_state',                 # state for Q1
    'org_city',                  # (optional) for detailed visuals
    'project_title', 'abstract', # for flagged words analysis
    'nsf_total_budget',          # main budget field (Q3)
    'status',                    # cancelled vs reinstated
    'termination_date',          # when it was terminated
    'reinstated',                # boolean
    'reinstatement_date'         # date
]

df_nsf_filtered = df_nsf[[c for c in keep if c in df_nsf.columns]].copy()

df_merged = pd.merge(df_nsf_filtered, df_cruz[['grant_id', 'in_cruz_list']], on='grant_id', how='left')

# flagged-word detection
import re

flagged_words = df_flagged['flagged_word'].dropna().str.lower().tolist()
pattern = r'\b(' + '|'.join(map(re.escape, flagged_words)) + r')\b'

# Combine project title + abstract, search for flagged terms
df_merged['has_flagged_word'] = (
    (df_merged['project_title'].fillna('') + ' ' + df_merged['abstract'].fillna(''))
    .str.lower()
    .str.contains(pattern, regex=True, na=False)
)


display(df_merged.head())


C:\Users\matil\AppData\Local\Temp\ipykernel_23044\1233843377.py:37: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  .str.contains(pattern, regex=True, na=False)


,grant_id,org_name,org_state,org_city,project_title,abstract,nsf_total_budget,status,termination_date,reinstated,reinstatement_date,in_cruz_list,has_flagged_word
0,2201796,Carnegie-Mellon University,PA,PITTSBURGH,Collaborative Research: Investigating Gender D...,Despite evidence that gender differences in ma...,1051218,0,2025-04-18,False,NaN,True,True
1,2315095,University of Texas at San Antonio,TX,SAN ANTONIO,Rhetorical Engineering Education to Support Pr...,This project will provide a handbook of materi...,3641116,0,2025-04-18,False,NaN,True,True
2,2236163,University of New Mexico,NM,ALBUQUERQUE,"Increasing the Effectiveness of Justice, Equit...",This project aims to serve the national intere...,609429,0,2025-04-18,False,NaN,True,True
3,2243822,University of California-Berkeley,CA,BERKELEY,NRT-HDR: Computational Research for Equity in ...,The criminal legal system is an important driv...,3000000,1,2025-04-18,True,2025-06-30,True,True
4,2216648,Oklahoma State University,OK,STILLWATER,RaMP: Oklahoma Network addressing human impact...,Building a stronger and more diverse scientifi...,2719700,0,2025-04-18,False,NaN,True,True


Another step that is done in order to futher decide how to treat the initial data, is to diplay information of each file, containing the non-null count, the type of each row and the range of the data. 

In [6]:
#Data information 
print("\nCruz data info:")
print(df_merged.info())
print(df_merged.isna().sum())



Cruz data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1970 entries, 0 to 1969
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   grant_id            1970 non-null   int64 
 1   org_name            1970 non-null   object
 2   org_state           1970 non-null   object
 3   org_city            1969 non-null   object
 4   project_title       1970 non-null   object
 5   abstract            1969 non-null   object
 6   nsf_total_budget    1970 non-null   int64 
 7   status              1970 non-null   int64 
 8   termination_date    1970 non-null   object
 9   reinstated          1970 non-null   bool  
 10  reinstatement_date  420 non-null    object
 11  in_cruz_list        1039 non-null   object
 12  has_flagged_word    1970 non-null   bool  
dtypes: bool(2), int64(3), object(8)
memory usage: 173.3+ KB
None
grant_id                 0
org_name                 0
org_state                0
org_city 

In [7]:
print("\nFlagged words data info:")
print(df_flagged.info())
print(df_flagged.isna().sum())



Flagged words data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   flagged_word  54 non-null     object
dtypes: object(1)
memory usage: 564.0+ bytes
None
flagged_word    0
dtype: int64


### Question 1: How are the cancellations distributed by states? 

In [8]:
# Count terminations by state
state_counts = (
    df_merged[df_merged['status'] == 0]['org_state']
    .value_counts()
    .reset_index()
)
state_counts.columns = ['state', 'Terminations']

# Select top 10 states
top10_states = state_counts.head(10)


alt.Chart(top10_states).mark_bar(color='steelblue').encode(
    y=alt.Y('state:N', sort='-x', title='state'),
    x=alt.X('Terminations:Q', title='Number of Cancellations')
).properties(
    title='Top 10 States With Most Cancellations'
)

alt.Chart(...)

In [9]:
state_counts.head()

,state,Terminations
0,MA,250
1,TX,118
2,NY,98
3,CA,80
4,FL,58


In [ ]:
#@title Top 10 States by NSF Terminations (minimal axes only)

import altair as alt
import pandas as pd

state_counts = (
    df_merged[df_merged['status'] == 0]['org_state']
    .value_counts()
    .reset_index()
)
state_counts.columns = ['state', 'Terminations']

top10_states = state_counts.head(10)

base = alt.Chart(top10_states, title='Top 10 States by NSF Terminations').encode(
    y=alt.Y('state:N', sort='-x', title='State', axis=alt.Axis(grid=False)),
    x=alt.X('Terminations:Q', title='Terminations', axis=alt.Axis(grid=False))
)

bars = base.mark_rule(color='lightgray').encode(
    x='Terminations:Q',
    x2=alt.value(0)
)

points = base.mark_circle(color='black', size=80)

text = base.mark_text(
    align='left',
    dx=6,
    fontSize=12
).encode(
    text='Terminations:Q'
)

chart = bars + points + text

chart.configure_axis(
    grid=False
).configure_view(
    stroke=None  # removes the top and right border lines
)


alt.LayerChart(...)

In [78]:
#@title Top 15 States by NSF Terminations (no size legend)

state_counts = (
    df_merged[df_merged['status'] == 0]['org_state']
    .value_counts()
    .reset_index()
)
state_counts.columns = ['state', 'Terminations']

top10_states = state_counts.head(15).reset_index(drop=True)

max_x = int(top10_states['Terminations'].max())
pad   = max(8, int(0.06 * max_x))

base = alt.Chart(top10_states, title='Top 15 States by NSF Terminations').encode(
    y=alt.Y('state:N', sort='-x', title=None,
            axis=alt.Axis(grid=False, labelFontSize=12)),
    x=alt.X('Terminations:Q', title=None,
            scale=alt.Scale(domain=[0, max_x + pad]),
            axis=alt.Axis(
                grid=False,
                labels=False,
                ticks=False,
                domain=True
            ))
)

bars = base.mark_rule(color='lightgrey').encode(
    x='Terminations:Q',
    x2=alt.value(0)
)

points = base.mark_circle(opacity=0.95, color='firebrick').encode(
    size=alt.Size('Terminations:Q',
                  legend=None,  # removes the size legend
                  scale=alt.Scale(type='sqrt', range=[30, 800]))
)

text = base.mark_text(
    align='left',
    dx=15,
    fontSize=12,
    color='black'
).encode(
    text=alt.Text('Terminations:Q', format='.0f')
)

chart = bars + points + text

chart.properties(
    width=640,
    height={'step': 30},
    padding={'right': 30}
).configure_axis(
    grid=False
).configure_view(
    stroke=None
).configure_title(
    fontSize=16,
    anchor='start',
    offset=20,
    
)


alt.LayerChart(...)

In [51]:
from vega_datasets import data as vega

#Aggregate: cancellations per state (distinct grants recommended) 
term = (
    df_merged[df_merged['status'] == 0]
    .groupby('org_state')['grant_id']
    .nunique()   # use .count() if each row is already one grant
    .reset_index(name='Terminations')
)

#  Map state abbreviations -> full names (for joining to lat/lon) 
abbr2name = {
    'AL':'Alabama','AK':'Alaska','AZ':'Arizona','AR':'Arkansas','CA':'California','CO':'Colorado',
    'CT':'Connecticut','DE':'Delaware','FL':'Florida','GA':'Georgia','HI':'Hawaii','ID':'Idaho',
    'IL':'Illinois','IN':'Indiana','IA':'Iowa','KS':'Kansas','KY':'Kentucky','LA':'Louisiana',
    'ME':'Maine','MD':'Maryland','MA':'Massachusetts','MI':'Michigan','MN':'Minnesota','MS':'Mississippi',
    'MO':'Missouri','MT':'Montana','NE':'Nebraska','NV':'Nevada','NH':'New Hampshire','NJ':'New Jersey',
    'NM':'New Mexico','NY':'New York','NC':'North Carolina','ND':'North Dakota','OH':'Ohio','OK':'Oklahoma',
    'OR':'Oregon','PA':'Pennsylvania','RI':'Rhode Island','SC':'South Carolina','SD':'South Dakota',
    'TN':'Tennessee','TX':'Texas','UT':'Utah','VT':'Vermont','VA':'Virginia','WA':'Washington',
    'WV':'West Virginia','WI':'Wisconsin','WY':'Wyoming','DC':'District of Columbia'
}
term['state_name'] = term['org_state'].map(abbr2name)

# Load US map + state capital coordinates from vega_datasets 
us = vega.us_10m.url
capitals = vega.us_state_capitals.url   # fields: state, lat, lon

# Prepare a dataframe Altair can use for the points by joining coordinates
# We'll do the join inside Vega-Lite via transform_lookup (keeps code tidy).
points = alt.Chart(term).transform_lookup(
    lookup='state_name',
    from_=alt.LookupData(capitals, key='state', fields=['state','lat','lon'])
).transform_filter(
    alt.datum.lat != None  # drop states that didn't match (e.g. None)
)

# Layers: base map + sized dots 
# Base map
background = alt.Chart(alt.Data(url=us, format={'type':'topojson','feature':'states'})) \
    .mark_geoshape(fill='#f5f5f5', stroke='#999')

# Dots sized by terminations
dots = points.mark_circle(color='steelblue', opacity=0.8).encode(
    longitude='lon:Q',
    latitude='lat:Q',
    size=alt.Size('Terminations:Q', title='Terminated grants',
                  scale=alt.Scale(range=[20, 1500])),  # tune bubble sizes
    tooltip=[
        alt.Tooltip('state_name:N', title='State'),
        alt.Tooltip('org_state:N', title='Abbr'),
        alt.Tooltip('Terminations:Q', title='Terminations')
    ]
)

# Compose
(background + dots).project(type='albersUsa').properties(
    width=800,
    height=500,
    title='Terminated NSF Grants by State (bubble size = count)'
)


alt.LayerChart(...)

In [ ]:
#@title Terminated NSF Grants per state

map = alt.topo_feature(vega.us_10m.url, feature='states')

usChart = alt.Chart(map).mark_geoshape(
    fill='lightgray',
    stroke='white',
).properties(
    width=500,
    height=300
).project('albersUsa')

df = (
    df_merged[df_merged['status'] == 0]
    .groupby('org_state')['grant_id']
    .nunique()
    .reset_index(name='Terminations')
)

abbr2name = {
    'AL':'Alabama','AK':'Alaska','AZ':'Arizona','AR':'Arkansas','CA':'California','CO':'Colorado',
    'CT':'Connecticut','DE':'Delaware','FL':'Florida','GA':'Georgia','HI':'Hawaii','ID':'Idaho',
    'IL':'Illinois','IN':'Indiana','IA':'Iowa','KS':'Kansas','KY':'Kentucky','LA':'Louisiana',
    'ME':'Maine','MD':'Maryland','MA':'Massachusetts','MI':'Michigan','MN':'Minnesota','MS':'Mississippi',
    'MO':'Missouri','MT':'Montana','NE':'Nebraska','NV':'Nevada','NH':'New Hampshire','NJ':'New Jersey',
    'NM':'New Mexico','NY':'New York','NC':'North Carolina','ND':'North Dakota','OH':'Ohio','OK':'Oklahoma',
    'OR':'Oregon','PA':'Pennsylvania','RI':'Rhode Island','SC':'South Carolina','SD':'South Dakota',
    'TN':'Tennessee','TX':'Texas','UT':'Utah','VT':'Vermont','VA':'Virginia','WA':'Washington',
    'WV':'West Virginia','WI':'Wisconsin','WY':'Wyoming','DC':'District of Columbia'
}
df['state_name'] = df['org_state'].map(abbr2name)

capitals = vega.us_state_capitals.url

terminationsMap = alt.Chart(df, title='Terminated NSF Grants distributed by State'
).transform_lookup(
    lookup='state_name',
    from_=alt.LookupData(capitals, key='state', fields=['state','lat','lon'])
).transform_filter(
    alt.datum.lat != None
).mark_circle(opacity=0.8).encode(
    longitude='lon:Q',
    latitude='lat:Q',
    size=alt.Size('Terminations:Q', title='Terminations', scale=alt.Scale(type='sqrt', range=[10, 1000])),
    color=alt.Color('Terminations:Q', title='Terminations', scale=alt.Scale(scheme='reds')),
    tooltip=['state_name:N','org_state:N','Terminations:Q']
)

usChart + terminationsMap


alt.LayerChart(...)

In [ ]:

# count DISTINCT terminated grants per state ---
state_counts = (
    df_merged[df_merged['status'] == 0]
    .groupby('org_state')['grant_id'].nunique()   # use .count() if already one row per grant
    .reset_index(name='Terminations')
    .sort_values('Terminations', ascending=False)
)

# top N
n = 20
top_states = state_counts.head(n)

# --- chart: horizontal bars, graded color, sorted by value ---
alt.Chart(top_states).mark_bar().encode(
    y=alt.Y('org_state:N', sort='-x', title='Top States'),
    x=alt.X('Terminations:Q', title='Number of Cancellations'),
    color=alt.Color('Terminations:Q', scale=alt.Scale(scheme='reds'), title='Terminations'),
    tooltip=['org_state:N', 'Terminations:Q']
).properties(
    width=500,
    height=300,
    title=f"Top {n} States by Terminations"
)


alt.Chart(...)

### Question 2:  What are the institutions that have been more affected in terms of number of cancelled grants. How does this compare to the others? 

In [13]:
# Count terminations by institution
inst_counts = (
    df_merged[df_merged['status'] == 0]['org_name']
    .value_counts()
    .reset_index()
)

# Rename columns manually (for pandas < 2.0)
inst_counts.columns = ['Institution', 'Terminations']

# Top 15 institutions
top_inst = inst_counts.head(15)

# Plot
import altair as alt

alt.Chart(top_inst).mark_bar(color='steelblue').encode(
    y=alt.Y('Institution:N', sort='-x', title='Institution'),
    x=alt.X('Terminations:Q', title='Number of Cancellations'),
    tooltip=['Institution:N', 'Terminations:Q']
).properties(
    width=780,
    title='Top 15 Institutions by Number of Cancellations'
)

# 

alt.Chart(...)

In [14]:
import altair as alt
import pandas as pd

# Count cancelled grants by institution name
institution_counts = (
    df_merged[df_merged['status'] == 0]['org_name']
    .value_counts()
    .reset_index()
)
institution_counts.columns = ['Institution', 'Cancelled Grants']

# Select top 10 institutions with most cancellations
top10_institutions = institution_counts.head(10)

# Visualize
alt.Chart(top10_institutions).mark_bar(color='indianred').encode(
    y=alt.Y('Institution:N', sort='-x', title='Institution'),
    x=alt.X('Cancelled Grants:Q', title='Number of Cancelled Grants'),
    tooltip=['Institution:N', 'Cancelled Grants:Q']
).properties(
    title='Top 10 Institutions With Most Cancelled Grants',
    width=300
)


alt.Chart(...)

In [80]:
#@title Top 15 Institutions by NSF Terminations (lollipop)

# Count terminations per institution
inst_counts = (
    df_merged[df_merged['status'] == 0]['org_name']
    .value_counts()
    .reset_index()
)
inst_counts.columns = ['Institution', 'Terminations']

# Top 15 institutions
top_inst = inst_counts.head(15).reset_index(drop=True)

# Headroom for labels on x-axis
max_x = int(top_inst['Terminations'].max())
pad   = max(8, int(0.06 * max_x))

base = alt.Chart(top_inst, title='Top 15 Institutions by NSF Terminations').encode(
    y=alt.Y('Institution:N', sort='-x', title=None,
            axis=alt.Axis(grid=False, labelFontSize=12, labelLimit=300)),
    x=alt.X('Terminations:Q', title=None,
            scale=alt.Scale(domain=[0, max_x + pad]),
            axis=alt.Axis(grid=False, labels=False, ticks=False, domain=True))
)

bars = base.mark_rule(color='lightgrey').encode(
    x='Terminations:Q',
    x2=alt.value(0)
)

points = base.mark_circle(opacity=0.95, color='firebrick').encode(
    size=alt.Size('Terminations:Q',
                  legend=None,
                  scale=alt.Scale(type='sqrt', range=[60, 700]))
)

text = base.mark_text(
    align='left',
    dx=15,
    fontSize=12,
    color='black'
).encode(
    text=alt.Text('Terminations:Q', format='.0f')
)

chart = bars + points + text

chart.properties(
    width=760,
    height={'step': 28},         # row height for 15 institutions
    padding={'top': 20, 'right': 40}
).configure_axis(
    grid=False
).configure_view(
    stroke=None
).configure_title(
    fontSize=16,
    anchor='start',
    offset=20
)


alt.LayerChart(...)

In [82]:
#@title Top 10 States by NSF Terminations (minimal axes only)

# Count terminations per institution
inst_counts = (
    df_merged[df_merged['status'] == 0]['org_name']
    .value_counts()
    .reset_index()
)
inst_counts.columns = ['Institution', 'Terminations']

# Top 15 institutions
top_inst = inst_counts.head(15).reset_index(drop=True)

# Headroom for labels on x-axis
max_x = int(top_inst['Terminations'].max())
pad   = max(8, int(0.06 * max_x))

base = alt.Chart(top_inst, title='Top 15 Institutions by NSF Terminations').encode(
    y=alt.Y('Institution:N', sort='-x', title=None,
            axis=alt.Axis(grid=False, labelFontSize=12, labelLimit=300)),
    x=alt.X('Terminations:Q', title=None,
            scale=alt.Scale(domain=[0, max_x + pad]),
            axis=alt.Axis(grid=False, labels=False, ticks=False, domain=True))
)

bars = base.mark_rule(color='lightgray').encode(
    x='Terminations:Q',
    x2=alt.value(0)
)

points = base.mark_circle(color='black', size=80)

text = base.mark_text(
    align='left',
    dx=6,
    fontSize=12
).encode(
    text='Terminations:Q'
)

chart = bars + points + text

chart.configure_axis(
    grid=False
).configure_view(
    stroke=None  # removes the top and right border lines
)


alt.LayerChart(...)

### Question 3: What are the institutions that have been more affected in terms of budget, and how does this compare to the others? 

In [81]:

# Ensure budget is numeric
tmp = df_merged.copy()
tmp['nsf_total_budget'] = pd.to_numeric(tmp['nsf_total_budget'], errors='coerce')

# Sum cancelled budget per institution
inst_budget = (
    tmp[tmp['status'] == 0]
    .groupby('org_name', dropna=False)['nsf_total_budget']
    .sum()
    .reset_index(name='BudgetCancelled')
    .sort_values('BudgetCancelled', ascending=False)
)

# Top 15 by budget
top_budget = inst_budget.head(15)

# Plot
alt.Chart(top_budget).mark_bar(color='steelblue').encode(
    y=alt.Y('org_name:N', sort='-x', title='Institution'),
    x=alt.X('BudgetCancelled:Q', title='Total Cancelled Budget (USD)', axis=alt.Axis(format='~s')),
    tooltip=[
        alt.Tooltip('org_name:N', title='Institution'),
        alt.Tooltip('BudgetCancelled:Q', title='Budget', format=',')
    ]
).properties(
    width=300,
    title='Top 15 Institutions by Cancelled Budget'
)


alt.Chart(...)

In [85]:
alt.Chart(top15, title='Top 15 Institutions: Terminations and Budget').mark_bar().encode(
    y=alt.Y('Institution:N', sort='-x', title='Institution', axis=alt.Axis(labelLimit=250)),
    x=alt.X('Terminations:Q', title='Terminations'),
    color=alt.Color('Budget:Q', title='NSF Total Budget', scale=alt.Scale(scheme='reds')),
    tooltip=['Institution', 'Terminations', alt.Tooltip('Budget:Q', format='$,.0f')]
).properties(width=600, height={'step': 28})


alt.Chart(...)

In [95]:
#@title Top 15 Institutions by NSF Terminations (color = Budget, no gridlines)


df = (
    df_merged[df_merged['status'] == 0]
    .groupby('org_name', as_index=False)
    .agg(Terminations=('grant_id', 'nunique'),
         Budget=('nsf_total_budget', 'sum'))
    .sort_values('Terminations', ascending=False)
    .head(15)
    .rename(columns={'org_name': 'Institution'})
)

alt.Chart(df, title='Top 15 Institutions by NSF Terminations & Budget').mark_circle(opacity=0.9).encode(
    x=alt.X('Terminations:Q', title='Terminations',
            axis=alt.Axis(grid=False)),       # remove x grid
    y=alt.Y('Institution:N', sort='-x', title=None,
            axis=alt.Axis(grid=False, labelLimit=300)),  # remove y grid
    size=alt.Size('Terminations:Q', legend=None,
                  scale=alt.Scale(type='sqrt', range=[60, 700])),
    color=alt.Color('Budget:Q', title='NSF Total Budget',
                    scale=alt.Scale(scheme='cividis')),  # colorblind-safe
    tooltip=['Institution', 'Terminations', alt.Tooltip('Budget:Q', format='$,.0f')]
).properties(
    width=600,
    height={'step': 28}
).configure_view(
    stroke=None  # removes top/right border
)


alt.Chart(...)

### Question 4: Is there any correlation between the cancelled grants and the list of flagged words? 

In [110]:
#@title Termination rate by flagged words (two bars, clean minimal style)

# Data prep
df_rate = (
    df_merged.assign(
        Flagged=np.where(df_merged['has_flagged_word'].astype(bool),
                         'Flagged', 'Not flagged'),
        Cancelled=(df_merged['status'] == 0).astype(int)
    )
    .groupby('Flagged', as_index=False)
    .agg(CancelRate=('Cancelled', 'mean'))  # proportion terminated
)

alt.Chart(df_rate, title='Termination Rate by Flagged Words').mark_bar(size=80).encode(
    x=alt.X('Flagged:N', title=None, axis=alt.Axis(labelFontSize=13)),
    y=alt.Y('CancelRate:Q', title='Share of grants terminated',
            axis=alt.Axis(format='.0%', labelFontSize=12)),
    color=alt.Color('Flagged:N',
                    title='Flagged word',
                    scale=alt.Scale(domain=['Flagged','Not flagged'],
                                    range=['#008080','#B0B0B0'])),  # colorblind-safe
    tooltip=[alt.Tooltip('CancelRate:Q', format='.1%')]
).properties(
    width=250,
    height=300
).configure_view(
    stroke=None
).configure_axis(
    grid=False
)


alt.Chart(...)

### Question 5:  Is there any correlation between the cancelled grants and the list of grants in Cruz’s list? And with respect to reinstated grants?

In [117]:
#@title Cancellation rate by Cruz's list (teal–gray style)

import altair as alt
import pandas as pd

# --- Data prep ---
cancel_rate = (
    df_merged.groupby('in_cruz_list')['status']
    .apply(lambda s: (s == 0).mean())
    .reset_index(name='CancelRate')
)
cancel_rate['Cruz'] = cancel_rate['in_cruz_list'].map({
    True: 'In Cruz list',
    False: 'Not in Cruz list'
})

# --- Base chart ---
base = alt.Chart(cancel_rate, title='Cancellation Rate: In vs. Not in Cruz’s List').encode(
    x=alt.X('Cruz:N', title=None, axis=alt.Axis(labelFontSize=13)),
    y=alt.Y('CancelRate:Q',
            title='Share of grants cancelled',
            axis=alt.Axis(format='.0%', labelFontSize=12))
)

# --- Bars ---
bars = base.mark_bar(size=120, opacity=0.9).encode(
    color=alt.Color('Cruz:N',
                    scale=alt.Scale(
                        domain=['In Cruz list', 'Not in Cruz list'],
                        range=['#008080', '#B0B0B0']  # Teal and warm gray
                    ),
                    legend=None)
)

# --- Text labels above bars ---
text = base.mark_text(
    align='center',
    dy=-8,
    fontSize=13,
    color='black'
).encode(
    text=alt.Text('CancelRate:Q', format='.0%')
)

# --- Combine and style ---
chart = (bars + text).properties(
    width=260,     # balanced width for 2 bars
    height=350,
    padding={'right': 20}
).configure_axis(
    grid=False
).configure_view(
    stroke=None
).configure_title(
    fontSize=16,
    anchor='start',
    offset=10
)

chart


alt.LayerChart(...)

In [122]:
#@title Cancellation & Reinstatement rates by Cruz's list (teal–gray, final version)

import altair as alt
import pandas as pd

# --- Cancellation rate ---
cancel_rate = (
    df_merged.groupby('in_cruz_list')['status']
    .apply(lambda s: (s == 0).mean())
    .reset_index(name='Rate')
)
cancel_rate['Group'] = cancel_rate['in_cruz_list'].map({
    True: 'In Cruz list',
    False: 'Not in Cruz list'
})

cancel_chart = (
    alt.Chart(cancel_rate, title='Cancellation Rate')
    .mark_bar(size=120, opacity=0.9)
    .encode(
        x=alt.X('Group:N', title=None, axis=alt.Axis(labelFontSize=13)),
        y=alt.Y('Rate:Q', title='Share cancelled',
                axis=alt.Axis(format='.0%', labelFontSize=12)),
        color=alt.Color('Group:N', legend=None,
                        scale=alt.Scale(domain=['In Cruz list', 'Not in Cruz list'],
                                        range=['#008080', '#B0B0B0']))
    )
    + alt.Chart(cancel_rate)
    .mark_text(align='center', dy=-8, fontSize=13, color='black')
    .encode(x='Group:N', y='Rate:Q', text=alt.Text('Rate:Q', format='.0%'))
).properties(width=260, height=350)

# --- Reinstatement rate (among cancelled) ---
reinst_rate = (
    df_merged[df_merged['status'] == 0]
    .groupby('in_cruz_list')['reinstated']
    .mean()
    .reset_index(name='Rate')
)
reinst_rate['Group'] = reinst_rate['in_cruz_list'].map({
    True: 'In Cruz list',
    False: 'Not in Cruz list'
})

reinst_chart = (
    alt.Chart(reinst_rate, title='Reinstatement Rate (among cancelled)')
    .mark_bar(size=120, opacity=0.9)
    .encode(
        x=alt.X('Group:N', title=None, axis=alt.Axis(labelFontSize=13)),
        y=alt.Y('Rate:Q', title='Share reinstated',
                axis=alt.Axis(format='.0%', labelFontSize=12)),
        color=alt.Color('Group:N', legend=None,
                        scale=alt.Scale(domain=['In Cruz list', 'Not in Cruz list'],
                                        range=['#008080', '#B0B0B0']))
    )
    + alt.Chart(reinst_rate)
    .mark_text(align='center', dy=-8, fontSize=13, color='black')
    .encode(x='Group:N', y='Rate:Q', text=alt.Text('Rate:Q', format='.0%'))
).properties(width=260, height=350)

# --- Combine side-by-side ---
final = alt.hconcat(cancel_chart, reinst_chart).configure_axis(
    grid=False
).configure_view(
    stroke=None
).configure_title(
    fontSize=16,
    anchor='start',
    offset=10
)

#final
df_merged['reinstated'].value_counts()



reinstated
False    1550
True      420
Name: count, dtype: int64